In [89]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils

In [90]:
DATA_DIR = "../../data/fb15k/"
DUMP_FILE = "../dumps/fb15k_distmult_dump_norm.pkl"
MODEL_TYPE = "distmult"
mining_dir='mined'

In [91]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [92]:
dump=utils.load_pickle(DUMP_FILE)
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [93]:
model=models.TypedDM(DUMP_FILE)

In [94]:
mapped_train_data = utils.map_data(train_data,dump)
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

In [95]:
entity_to_rel=utils.get_ent_to_rel(mapped_train_data)

In [111]:
rules_1_path=os.path.join(mining_dir,"pruned_rules_1.pkl")
rules_2_path=os.path.join(mining_dir,"pruned_rules_2.pkl")
rules_1=utils.load_pickle(rules_1_path)
rules_2=utils.load_pickle(rules_2_path)
print(len(rules_1),len(rules_2))

3869 1836


In [125]:
def add_relation_body(rules,relation_to_body):
    for rule in rules:
        if rule[1] not in relation_to_body:
            relation_to_body[rule[1]]=[]
        relation_to_body[rule[1]].append(rule[0])
    return relation_to_body

In [126]:
relation_to_body={}
relation_to_body=add_relation_body(rules_1,relation_to_body)
relation_to_body=add_relation_body(rules_2,relation_to_body)
print(len(relation_to_body))

919


# Length 1 Rules

In [127]:
dict_1=utils.get_r_e1e2_dict(mapped_train_data)

# Length 2 Rules

In [128]:
index_head=utils.get_head_index(mapped_train_data)
dict_2=utils.get_r1r2_e1e2_dict(mapped_train_data,index_head)

In [129]:
def get_explanation(fact,relation_to_body,dict_1,dict_2):
    pair=(fact[0],fact[2])
    rel=fact[1]
    
    if rel not in relation_to_body:
        return ("",-1)
    bodies=relation_to_body[rel]
    for body in bodies:
        if isinstance(body,int):
            if pair not in dict_1[body]:
                continue
            return (body,-1)
        else:
            if pair not in dict_2[body]:
                continue
            return (body,dict_2[body][pair])
    return ("",-1)

In [130]:
# get_explanation((453,37,82),relation_to_body,dict_1,dict_2)

In [145]:
triplet_file="../logs/fb15k/turk_test/test_hits_1_id_small.txt"
data=utils.read_data(triplet_file)
np_arr=np.array(utils.map_data(data,dump=dump)).astype(np.int32)
# np_arr=np.loadtxt(triplet_file)

In [146]:
arr=[]
for line in np_arr:
#     print(line)
    fact=(int(line[0]),int(line[1]),int(line[2]))
    arr.append(get_explanation(fact,relation_to_body,dict_1,dict_2))
#     print(fact)

In [147]:
# print(arr)
count=0
for x in arr:
    if x[0]!="":
        count+=1
print(count)

85


In [148]:
path="./mined/explanation.pkl"
utils.dump_pickle(arr,path)

In [149]:
print(len(np_arr),len(arr))

200 200


In [138]:
lol=utils.load_pickle(path)

In [140]:
print(len(lol))
print(lol[0:100])

37871
[(164, -1), ('', -1), ('', -1), (164, -1), ('', -1), ('', -1), (61, -1), (4, -1), ('', -1), ('', -1), (173, -1), (278, -1), ('', -1), (28, -1), (42, -1), ('', -1), ('', -1), ('', -1), ('', -1), (291, -1), (164, -1), (473, -1), (280, -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), (64, -1), (64, -1), (970, -1), ('', -1), ('', -1), ('', -1), ('', -1), (345, -1), ('', -1), ('', -1), ('', -1), ('', -1), (23, -1), (64, -1), ('', -1), ('', -1), (164, -1), (50, -1), (50, -1), (44, -1), ('', -1), (10, -1), ('', -1), (184, -1), ('', -1), (420, -1), ('', -1), ('', -1), (305, -1), ('', -1), ('', -1), ('', -1), (98, -1), ('', -1), (53, -1), ('', -1), ('', -1), (53, -1), (61, -1), (61, -1), (119, -1), ('', -1), ('', -1), ('', -1), ('', -1), (461, -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ('', -1), ((37, 11), 1871), ('', -1), (437, -1), ('', -1), (50, -1), (35, -1), (99, -1), ('', -1),